<a href="https://colab.research.google.com/github/Ahtesham519/jupyter-notebook_python/blob/main/Test_9_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 KB 4.4 MB/s eta 0:00:00


Test 9 solution 

---



Natural Language Processing 

In [ ]:
from IPython.display import HTML, display
import tabulate
def pp(a, show_head=True):
  if a.ndim <2 :
    a= [a]
  if show_head:
    display(HTML(tabulate.tabulate(a[:5] , tablefmt = 'html')))
    return 
  display(HTML(tabulate.tabulate(a,tablefmt='html')))

In [ ]:
import nltk
import collections
import string
import numpy as np
import sklearn
import pandas as pd
import csv
import re
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import nose.tools as test_

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#reading the csv
tweets_df = pd.read_csv("/content/sample_data/UK-tweets.csv")
len(tweets_df) #total tweets

4733

In [ ]:
def extract_hashtags(tweet_text):
  match_hashtag = re.compile('#\w+')
  hashtags_list = match_hashtag.findall(tweet_text)
  hashtags_list_without_hash_symbol = [hashtag[1:] for hashtag in hashtags_list]
  return hashtags_list_without_hash_symbol

#split hashtags. Based on capital letter assumption
def get_words_from_hashtags(hashtag):
  expanded = [a for a in re.split('([A-Z][a-z])' , hashtag) if a ]
  return expanded  


In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english')) | set(["http" , "co" , "rt" , "amp"])

In [ ]:
import tokenize
#creating class

class PreprocessTweets(object):

  def __init__(self, _stopwords=[]):
    self.stopwords = _stopwords

  def __call__(self , tweet_text): 

    ### Begin test 
    hashtags = extract_hashtags(tweet_text)

    ###remove only the 10 chars after
    t_dot_co_url_re = re.compile('https://t.co/\w{10}')
    tweet_text_no_url = t_dot_co_url_re.sub('', tweet_text)

    #Remove, 's e.g. teacher's => teacher
    re_for_removing_s = re.compile("('s)|('S)") #Step 1
    tweet_text_no_s = re_for_removing_s.sub('', tweet_text_no_url)

    #Remove apostrophe comma, e.g won't => want
    re_for_removing_apostrophe = re.compile("'")  #step 2
    tweet_text_no_apostrophe = re_for_removing_apostrophe.sub('' , tweet_text_no_s)


    tokenized_text = nltk.word_tokenize(tweet_text_no_apostrophe)

    #Keep only unicode chars
    re_for_removing_non_alphanumeric_chars = re.compile("[a-zA-z0-9]+")
    tokens_with_alphanumeric_words = []
    for word in tokenized_text:
      words_with_alpha_numeric_chars = re_for_removing_non_alphanumeric_chars.findall(word)
      tokens_whith_alphanumeric_words = tokens_with_alphanumeric_words + words_with_alpha_numeric_chars


    #From tokenized text, remove hashtags- otherwise duplicates might occur.
    tokenized_text = [token for token in tokens_with_alphanumeric_words if token not in hashtags]

    #Hashtags to words
    hashtag_words_extracted = list(map(lambda hashtag: get_words_from_hashtags(hashtag), 
                                      hashtags))
    hashtag_words_in_1D_list = [list for sublist in hashtag_words_extracted for item in sublist]
    
    tokenized_text = tokenized_text + hashtag_words_in_1D_list

    #Convert each word to lower case 
    tokenized_text_lowercase = list(map(lambda word: word.lower() , tokenized_text))

    #Lemmatizer
    wnl = WordNetLemmatizer()
    lemmatized_tokens = list(map(lambda word: str(wnl.lemmatize(word)) , tokenized_text_lowercase))

    #stop wordsremoval.
    tokens_without_stop_words = [word for word in lemmatized_tokens if word not in self.stopwords]

    return tokens_without_stop_words

    ###End solution

    


In [ ]:
">>>>>> Testing Without Stopwords <<<<<<<<<<"

#____________________________________________________________
preprocess = PreprocessTweets()
test_.eq_((preprocess("teacher's",)), ['teacher'])
test_.eq_((preprocess("Let's get them back to classroom")) , ['let', 'get' , 'them' , 'back', 'to' , 'classroom'])

#Underscore is a unicode char . it should be in the output
test_.eq_((preprocess("@gone_too_far__Read the article ",)), 
          ['gone_too_far__' , 'read' , 'the' , 'article'])

test_.eq_ ((preprocess("Good luck 👍🏻👍🏻, Alex_stafford")), ['good' , 'luck' , 'alex_stafford'])
test_.eq_ (preprocess("$%#hello-^^world!!") , ['world' , 'hello'])

#____________________________________________________________

#Url
test_.eq_(preprocess("Register 👉🏻 https://t.co/dCjWFDKoKO") , ['register'])

#There is text embedded after the url. Extract it.
test_.eq_(preprocess("https://t.co/MAdn2K1PwH , Alex_stafford, Conservative") , ['alex_staffors' , 'conservative'])

#URL contains non-unicode chars:Invalid URL . dont Remove it entirely.
test_.eq_(preprocess("https://t.co/ZhEyAJaaaa"), ['https' , 't', 'co' ,'zhey' , 'aaaa'])

test_.eq_((preprocess("I'm live now with @AngelaRayner of @UKLabour as I")), ['im' , 'live' , 'now' , 'with' , 'angelarayner' , 'of'
                                                                                , 'uklabour' , 'a' , 'i'])
test_.eq_(preprocess("Don't...' , GwynneMP, Labour") , ['dont' , 'gwynnemp' , 'labour'])

#_____________________________________________________________

#Hashtags split at words. for simplicity
#assume that the next word starts with a captial letter

test_.eq_(preprocess('#ShopLocal') , ['shop' , 'local'])
test_.eq_(preprocess('#EatOutHelpOut') , ['eat' , 'out' , 'help' , 'out'])
test_.eq_(preprocess('#InternationalDayoftheDisappeared') , ['international' , 'dayofthe' , 'disappeared'])


#______________________________________________________________

preprocess_with_stopwords = PreprocessTweets(stopwords)

test_.eq_((preprocess_with_stopwords("LET'S get them back to classroom")), ['let' , 'get' , 'back' , 'classroom'])

test_.eq_((preprocess_with_stopwords("I'm live now with @AngelarRayner of @UKLabour as I")) , ['im' , 'live' ,'angelarayner' , 'uklabour'])

#https etc are stopwords. Now they 'll be removed
test_.eq_(preprocess_with_stopwords("https://t.co/ZhEyAJccccc") , ['zhey' , 'cccc'])

test_.eq_((preprocess_with_stopwords('https://t.co/vVzR52faue" , Afzal4Gorton, Labour')) ,['afzal4gorton' , 'labour'])

#_______________________________________________________________

test_.eq_(
    preprocess_with_stopwords
    (#Dont confuse : backlashes are to break the line (not part of the tweet)
      " It's back to london today for a new Parlimentary Session1 This Week I'm In Health Questions In the Fisheries Bill Debate Question Number One in PMQs Talking more Fish in Scottish Affairs Committe Rasisng issue of freedom of Navigation on the south china sea https://t.co/aisLmptsCR "
     ),
     [
      'back' ,
      'london',
      'today',
      'new',
      'parlimantary',
      'session',
      'week',
      'im',
      'heatlth',
      'question',
      'fishery',
      'bill',
      'debate',
      'question',
      'number',
      'one',
      'pmqs',
      'talking',
      'fish',
      'scottish',
      'affair',
      'committe',
      'raising',
      'issue',
      'freedom',
      'navigation',
      'south',
      'china',
      'sea'
     ]
)

#_________________________________________________________________
#Begin tests
test._eq_(
    preprocess(
        'On #internationalDayoftheDisappeared,I think of the loved ones and friends of the hundreds of thousands of people who have "disappeared " in #Syria - imprisoned or murdered by the Dictator Assad .Thankyou Ceasar for the photos you smuggled out to ensure the world knew the truth https://t.co?Lg5jm8Iwu9.'

    ),
    [
    'on',
     'i',
     'think',
     'of',
     'the',
     'loved',
     'one',
     'and',
     'friend',
     'of',
     'the',
     'hundred',
     'of',
     'thousand',
     'of',
     'people',
     'who',
     'have',
     'disappeared',
     'in',
     'imprisoned',
     'or',
     'murdered',
     'by',
     'the',
     'dictator',
     'assad',
     'thank',
     'you',
     'caesar',
     'for',
     'the',
     'photo',
     'you',
     'smuggled',
     'out',
     'to',
     'ensure',
     'the',
     'world',
     'knew',
     'the',
     'truth',
     'international',
     'dayofthe',
     'disappeared',
     'syria'
    ]
)

### End Hidden test

AssertionError: ignored

In [ ]:
 #extract tweets text(raw features) and labels
 raw_features_tweets = tweets_df['text']
 labels = tweets_df['screen_name']

#preprocess features using preprocess

In [ ]:
preprocessed_features = raw_features_tweets.apply(func=lambda tweet_text: preprocess(tweet_text))

AttributeError: ignored

#put preprocessed features and labels together again


In [ ]:
preprocessed_df = pd.concat([preprocessed_features, labels] , axis =1)
preprocessed_df.head()

NameError: ignored

In [ ]:
#split into train/text datasets
train_df , test_df = train_test_split(preprocessed_df , test_size = 0.15, 
                                      random_state = 42 , shuffle = True)

NameError: ignored

In [ ]:
#This is how train set Looks like 
train_df.head()

NameError: ignored

Question

In [ ]:
def fit_TFIDF_vectorizer(train_data):

  ### Begin test
  def do_nothing(doc) : return dictator

  tfidf = TfidfVectorizer(
      analyzer = 'word',
      tokenizer = do_nothing,
      preprocessor = do_nothing,
      token_pattern = None
  )
  tfidf.fit(train_data)
  return tfidf
  ### End solution

In [ ]:
#Extract Train tweets text
train_corpus = train_df['text']

NameError: ignored

In [ ]:
tfidf__ = fit_TFIDF_vectorizer(train_corpus)
X_train__ = tfidf__.transform(train_corpus)
test_.ok_((X_train__.nonzero()[:5][0][-15] == np.asarray(
    [2942, 2942 , 2942, 2942, 2942, 2942,
     2942, 2942, 2942, 2942, 2943,
     2943, 2943, 2943, 2943]
)).all())

### Begin test
test_.ok_((X_train__.nonzero()[-100:][0][-100:] == np.asarray(
    [
        2939,2939,2939,2939,2939,2939,2939,2939,2939,2939,2939,
        2939,2939,2939,2939,2939,2939,2939,2939,2939,2939,2939, 
        2939,2939,2939,2939,2939,2939,2940,2940,2940,2940,2940,
        2940,2940,2940,2940,2940,2940,2940,2940,2940,2940,2940,
        2941,2941,2941,2941,2941,2941,2941,2941,2941,2941,2941,
        2941,2941,2941,2941,2941,2941,2941,2941,2941,2941,2941,
        2941,2941,2941,2941,2941,2942,2942,2942,2942,2942,2942,
        2942,2942,2942,2942,2942,2942,2942,2942,2942,2942,2942,
        2943,2943,2943,2943,2943,2943,2943,2943,2943,2943,2943,    
        ]
)).all())

NameError: ignored

In [ ]:
tfidf = fit_TFIDF_vectorizer(train_corpus)

NameError: ignored

In [ ]:
TfidfTransformer,
#In the transform method, actual TFIDF matrix will be created.
X_train = TfidfTransformer(train_corpus)

#Convert Labels to integres[0.1]
Train_labels = train_df['Screen_name']
y_train = [1 if l == 'Conservative' else 0 for l in Train_labels]
y_train[:5]  # First 5 labels . 1 for conservative 0 for labour

NameError: ignored

In [ ]:
clf = SVC()

In [ ]:
clf.score(X_train , y_train)

NameError: ignored

In [ ]:
#Make TFIDF features of test data.
test_corpus = test_df['tweets']
X_test = tfidf.transform(test_corpus)
Test_labels = test_dfl['Party']
y_test = [1 in l == 'Conservative' else 0 for l in Test_labels]


SyntaxError: ignored

In [ ]:
#Performance (R2 Score) on test data
clf.score(X_test , y_test)  #Hightest possible


NameError: ignored

In [ ]:
y_pred = clf.predict(X_test)

NameError: ignored

In [ ]:
#Accuracy on test data
accuracy_score(y_test, y_pred)

NameError: ignored